# ML - Problem Set 2: Predicting Poverty

El objetivo principal es construir un modelo predictivo de la pobreza de los hogares. Nótese que un hogar se clasifica como

\begin{equation}
    Poor = I \left( Inc < Pl \right)
\end{equation}

donde $I$ es una función indicadora que toma uno si el ingreso familiar está por debajo de una determinada línea de pobreza.

In [1]:
# Importar librerias

import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [2]:
# Importar datos

df_1 = pd.read_csv('sample_submission.csv')
df_2 = pd.read_csv('test_hogares.csv')
df_3 = pd.read_csv('test_personas.csv')
df_4 = pd.read_csv('train_hogares.csv')
df_5 = pd.read_csv('train_personas.csv')

In [3]:
# Renombrar columnas antes de combinar
df_4 = df_4.rename(columns={'Clase': 'Clase_hogar', 'Dominio': 'Dominio_hogar'})
df_5 = df_5.rename(columns={'Clase': 'Clase_persona', 'Dominio': 'Dominio_persona'})

# Combinar datos de entrenamiento
train_combined = pd.merge(df_4, df_5, on='id', how='inner')

# Combinar datos de prueba
test_combined = pd.merge(df_2, df_3, on='id', how='inner')

print("Dimensiones del dataset combinado de entrenamiento:", train_combined.shape)
print("Dimensiones del dataset combinado de prueba:", test_combined.shape)

Dimensiones del dataset combinado de entrenamiento: (543109, 157)
Dimensiones del dataset combinado de prueba: (219644, 78)


In [4]:
# Verificar hogares sin personas asociadas y viceversa
hogares_sin_personas = df_4[~df_4['id'].isin(df_5['id'])]
personas_sin_hogares = df_5[~df_5['id'].isin(df_4['id'])]
print("Número de hogares sin personas asociadas:", len(hogares_sin_personas))
print("Número de personas sin hogares asociados:", len(personas_sin_hogares))

Número de hogares sin personas asociadas: 0
Número de personas sin hogares asociados: 0


In [5]:
# Guardar la variable objetivo antes de alinear columnas
y_train = train_combined['pobre'] if 'pobre' in train_combined.columns else None

In [6]:
# Identificar columnas comunes entre train_combined y test_combined
common_columns = train_combined.columns.intersection(test_combined.columns)

# Alinear datasets
train_combined = train_combined[common_columns]
test_combined = test_combined[common_columns]

# Verificar las dimensiones tras la alineación
print("Dimensiones tras alineación de train_combined:", train_combined.shape)
print("Dimensiones tras alineación de test_combined:", test_combined.shape)

# Identificar columnas numéricas y categóricas
numeric_columns = train_combined.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = train_combined.select_dtypes(include=['object']).columns

# Imputar valores faltantes
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

# Imputar en train_combined
train_combined[numeric_columns] = num_imputer.fit_transform(train_combined[numeric_columns])
train_combined[categorical_columns] = cat_imputer.fit_transform(train_combined[categorical_columns])

# Imputar en test_combined
test_combined[numeric_columns] = num_imputer.transform(test_combined[numeric_columns])
test_combined[categorical_columns] = cat_imputer.transform(test_combined[categorical_columns])

# Verificar valores faltantes
print("Valores faltantes en train_combined:", train_combined.isnull().sum().sum())
print("Valores faltantes en test_combined:", test_combined.isnull().sum().sum())

Dimensiones tras alineación de train_combined: (543109, 74)
Dimensiones tras alineación de test_combined: (219644, 74)
Valores faltantes en train_combined: 0
Valores faltantes en test_combined: 0


In [7]:
train_combined

,id,P5000,P5010,P5090,P5100,P5130,P5140,Nper,Npersug,Li,...,P7510s5,P7510s6,P7510s7,Pet,Oc,Des,Ina,Fex_c_y,Depto_y,Fex_dpto_y
0,8d0193e328305a042001a35f,4.0,2.0,1.0,500000.0,400000.0,400000.0,5.0,5.0,122809.534158,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,127.222038,5.0,146.251790
1,8d0193e328305a042001a35f,4.0,2.0,1.0,500000.0,400000.0,400000.0,5.0,5.0,122809.534158,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,127.222038,5.0,146.251790
2,8d0193e328305a042001a35f,4.0,2.0,1.0,500000.0,400000.0,400000.0,5.0,5.0,122809.534158,...,2.0,1.0,2.0,1.0,1.0,1.0,1.0,127.222038,5.0,146.251790
3,8d0193e328305a042001a35f,4.0,2.0,1.0,500000.0,400000.0,400000.0,5.0,5.0,122809.534158,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,127.222038,5.0,146.251790
4,8d0193e328305a042001a35f,4.0,2.0,1.0,500000.0,400000.0,400000.0,5.0,5.0,122809.534158,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,127.222038,5.0,146.251790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543104,5f037d9ddb23a0223d937b30,2.0,1.0,3.0,500000.0,350000.0,400000.0,4.0,4.0,123416.961118,...,2.0,1.0,2.0,1.0,1.0,1.0,1.0,386.721150,11.0,386.721150
543105,e575d8c70d587b3be3f45a6f,4.0,3.0,3.0,500000.0,350000.0,1150000.0,4.0,4.0,123416.961118,...,2.0,1.0,2.0,1.0,1.0,1.0,1.0,341.056945,11.0,341.056945
543106,e575d8c70d587b3be3f45a6f,4.0,3.0,3.0,500000.0,350000.0,1150000.0,4.0,4.0,123416.961118,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,341.056945,11.0,341.056945
543107,e575d8c70d587b3be3f45a6f,4.0,3.0,3.0,500000.0,350000.0,1150000.0,4.0,4.0,123416.961118,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,341.056945,11.0,341.056945


In [8]:
test_combined

,id,P5000,P5010,P5090,P5100,P5130,P5140,Nper,Npersug,Li,...,P7510s5,P7510s6,P7510s7,Pet,Oc,Des,Ina,Fex_c_y,Depto_y,Fex_dpto_y
0,2a7ddc2779480d7f19834953,4.0,3.0,1.0,500000.0,400000.0,400000.0,5.0,5.0,121449.452925,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,7.826095,47.0,7.528079
1,2a7ddc2779480d7f19834953,4.0,3.0,1.0,500000.0,400000.0,400000.0,5.0,5.0,121449.452925,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,7.826095,47.0,7.528079
2,2a7ddc2779480d7f19834953,4.0,3.0,1.0,500000.0,400000.0,400000.0,5.0,5.0,121449.452925,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,7.826095,47.0,7.528079
3,2a7ddc2779480d7f19834953,4.0,3.0,1.0,500000.0,400000.0,400000.0,5.0,5.0,121449.452925,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,7.826095,47.0,7.528079
4,2a7ddc2779480d7f19834953,4.0,3.0,1.0,500000.0,400000.0,400000.0,5.0,5.0,121449.452925,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,7.826095,47.0,7.528079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219639,569496964b70185785b57789,2.0,2.0,4.0,500000.0,300000.0,400000.0,5.0,5.0,121522.856805,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,16.470949,66.0,14.496007
219640,225c4a98fcfad97cf4b08d0f,3.0,2.0,3.0,500000.0,350000.0,320000.0,4.0,4.0,126599.775559,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,95.694378,76.0,123.446537
219641,225c4a98fcfad97cf4b08d0f,3.0,2.0,3.0,500000.0,350000.0,320000.0,4.0,4.0,126599.775559,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,95.694378,76.0,123.446537
219642,225c4a98fcfad97cf4b08d0f,3.0,2.0,3.0,500000.0,350000.0,320000.0,4.0,4.0,126599.775559,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,95.694378,76.0,123.446537
